In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from plotly.colors import sequential
from scipy.optimize import curve_fit
from scipy.stats import chi2

In [2]:
def read(filename):
    df = pd.read_csv(filename, skiprows = 2, nrows = 151, sep = ',', header = None)
    df.dropna(axis = 1, inplace = True)  # Drop columns with NaN values
    df.reset_index(drop = True, inplace = True)  # Reset index
    df.columns = ['w0', 'I0', 'w1', 'I1', 'w2', 'I2', 'w3', 'I3', 'w4', 'I4', 'w5', 'I5']
    return df

In [3]:
def plottiamo(data):
    fig = go.Figure()
    colors = sequential.Sunsetdark_r[:6]

    for i in range(6):
        fig.add_trace(go.Scatter(
            x = data[f'w{i}'], 
            y = data[f'I{i}'], 
            mode = 'lines', 
            name = f'{i}M',
            line = dict(color = colors[i])
        ))

    fig.update_layout(
        xaxis_title = "Wavelength (nm)",
        yaxis_title = "Intensity",
        template = "plotly_white",
        height = 500,
        width = 800,
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        )
    )

    fig.show()

In [4]:
file = 'fluo2/GuHCl5.csv'
data = read(file)
plottiamo(data)

In [5]:
def parabola(x, a, x0, yV):
    return -a * (x - x0)**2 + yV

In [6]:
# FIT

fit_results, fit_errors = {}, {}
VERTEX = []
errorVertex = []
SIGMA_LAMBDA = 1.5  # nm DICHIARATI DAL COSTRUTTORE
POINTS_AROUND_MAX = 10  # Numero di punti attorno al massimo per il fit

for i in range(6):
    w_col, I_col = f'w{i}', f'I{i}'
    max_idx = data[I_col].idxmax()
    l_idx = max_idx - POINTS_AROUND_MAX
    r_idx = max_idx + POINTS_AROUND_MAX

    # subset per il fit parabolico
    w_subset, I_subset = data[w_col].iloc[l_idx:r_idx + 1], data[I_col].iloc[l_idx:r_idx + 1]  # subset creato con gli indici nel range

    # Iterazione 0
    popt, pcov = curve_fit(parabola, w_subset, I_subset, p0=[1, w_subset.mean(), I_subset.max()])
    a, λcenter, IMAX = popt
    error_a, error_λcenter, error_IMAX = np.sqrt(np.diag(pcov))

    # Iterazione 1
    sy = np.full_like(w_subset, SIGMA_LAMBDA)*a  # Errore su x (sigma lambda)
    popt, pcov = curve_fit(parabola, w_subset, I_subset, p0=[a, λcenter, IMAX], sigma=sy)
    a, λcenter, IMAX = popt
    error_a, error_λcenter, error_IMAX = np.sqrt(np.diag(pcov))

    # Salvataggio dei risultati del fit
    fit_results[f'{i} M'] = popt
    fit_errors[f'{i} M'] = np.sqrt(np.diag(pcov))

    # Calcolo del vertice
    VERTEX.append([λcenter, IMAX])
    errorVertex.append([error_λcenter,error_IMAX])



# fit_results = pd.DataFrame(fit_results)
print(fit_results)


{'0 M': array([4.20802185e-01, 3.33274780e+02, 5.75792305e+02]), '1 M': array([5.47102862e-01, 3.33446735e+02, 6.49519720e+02]), '2 M': array([4.41440044e-01, 3.44718788e+02, 7.46069908e+02]), '3 M': array([4.76240040e-01, 3.54877823e+02, 6.49345047e+02]), '4 M': array([4.96232169e-01, 3.55621363e+02, 6.25731630e+02]), '5 M': array([4.91838832e-01, 3.55713550e+02, 6.68858279e+02])}


In [7]:

fig = go.Figure()

colors = sequential.Sunsetdark_r[:6]  # Colori per le serie

for i in range(6):
    w_col, I_col = f'w{i}', f'I{i}'

    fig.add_trace(go.Scatter( # plot full dashed spectrum
        x=data[w_col], y=data[I_col], mode='lines', name=f'{i} M',
        line=dict(color=colors[i], dash='dot')
    ))

    # plot parabola
    max_idx = data[I_col].idxmax()
    l_idx = max_idx - POINTS_AROUND_MAX
    r_idx = max_idx + POINTS_AROUND_MAX

    # subset per il fit parabolico
    w_subset, I_subset = data[w_col].iloc[l_idx:r_idx + 1], data[I_col].iloc[l_idx:r_idx + 1]  # subset creato con gli indici nel range
    x_parabola = np.linspace(w_subset.min(), w_subset.max(), 100)
    y_parabola = parabola(x_parabola, *fit_results[f'{i} M'])

    # Aggiunta della parabola
    fig.add_trace(go.Scatter(
        x=x_parabola, y=y_parabola, mode='lines', name=f'Parabola Fit (Series {i})',
        line=dict(color=colors[i]), showlegend=False
    ))

    # Vertice con barre d'errore
    xV,yV = VERTEX[i]
    fig.add_trace(go.Scatter(
        x=[xV], y=[yV], mode='markers', name=f'Vertex (Series {i})',
        marker=dict(color=colors[i], size=8, symbol='circle'),
        showlegend=False
    ))

# Configurazione del layout
fig.update_layout(
    xaxis_title="Wavelength (nm)",
    yaxis_title="Intensity",
    template="plotly_white",
    height=500,
    width=800,
    legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        )
)

fig.show()

# Stampa dei risultati del fit
for key, values in fit_results.items():
    errors = fit_errors[key]
    formatted_values = f"xV = {values[1]:.2f} ± {errors[1]:.2f}, yV = {values[2]:.2f} ± {errors[2]:.2f}"
    # print(f"{key}: {formatted_values}")


In [8]:
print(VERTEX)

[[np.float64(333.27478031152907), np.float64(575.7923054495121)], [np.float64(333.4467351537085), np.float64(649.5197202437809)], [np.float64(344.71878840101493), np.float64(746.0699080331476)], [np.float64(354.8778232667291), np.float64(649.3450465288554)], [np.float64(355.62136317741454), np.float64(625.7316301943634)], [np.float64(355.71354961075787), np.float64(668.8582786321076)]]


In [9]:
# Cden = concentrazione del denaturante -> valore x
# Cmp = concentrazione del campione -> parametro
# m = pendenza della curva -> parametro
# yN = intensità del campione non denaturato -> valore noto
# yD = intensità del campione denaturato -> valore noto


# udm [m] = J/ mol2 L

def sigmoide(Cden, Cmid, m): # SCUSA SARAAAAAA, SONO IO A NON SAPER LEGGERE, ORA è OK MA LO ERA ANCHE PRIMA
    R = 8.314 # J / mol K
    T = 298.15  # K
    yN = VERTEX[0][0] # nm
    yD = VERTEX[5][0] # nm
    expon = m * (Cmid - Cden) / (R * T)
    return (yN + yD * np.exp(-expon)) / (1 + np.exp(-expon))

In [10]:
# Estrai gli errori su xV dai dati esistenti
errors_xV = [fit_errors[f'{i} M'][1] for i in range(6)]  # Errori su xV per ogni serie
# Definizione di assex e assey dai dati dei vertici
assex = np.array([i for i in range(6)]) # Indici delle serie (0 M, 1 M, ..., 5 M)
assey = np.array([VERTEX[i][0] for i in range(6)])  # Valori di xV per ogni serie

# Fit dei dati usando la funzione sigmoide e considerando gli errori
popt1, pcov1 = curve_fit(sigmoide, assex, assey, sigma=errors_xV, p0=[2.12, -34.27], maxfev=5000)

Cmid, m = popt1
err_Cmid, err_m = np.sqrt(np.diag(pcov1))


print('Cmid: ', final_val(Cmid,err_Cmid,udm='M'))
print('m: ', final_val(m,err_m,udm='J L / mol2'))

ΔG = Cmid * m
err_DELTA_G = np.sqrt((err_Cmid * m)**2 + (Cmid * err_m)**2)
print('ΔG: ', final_val(ΔG, err_DELTA_G, udm='J/mol'))

Cmid:  1.99 ± 0.01 M
m:  8929.8 ± 736.97 J L / mol2
ΔG:  17784.78 ± 1470.95 J/mol


In [11]:
fig = go.Figure()

# Fit sigmoide
Cden_fit = np.linspace(min(assex), max(assex), 1000)  # Genera valori per l'asse x
sigmoide_fit = sigmoide(Cden_fit, *popt1)  # Calcola i valori della sigmoide

# Sigmoide grafico
fig.add_trace(go.Scatter(
    x=Cden_fit,
    y=sigmoide_fit,
    mode='lines',
    name='Fit Sigmoide',
    line=dict(color=colors[0]),
    showlegend=False
))

# Dati originali con barre d'errore
fig.add_trace(go.Scatter(
    x=assex,
    y=assey,
    mode='markers',
    name='Dati Originali',
    marker=dict(color=colors[2], size=5),
    error_y=dict(
        type='data',
        array=errors_xV,
        visible=True
    ),
    showlegend=False
))

fig.update_layout(
    xaxis_title="C denaturante (M)",
    yaxis_title="wavelength (nm)",
    template="plotly_white",
    height=500,
    width=800    
)

fig.show()

In [12]:

y_fit = sigmoide(np.array(assex), *popt1)
errors_y = errors_xV  # Errori associati ai dati osservati

ChiQ = np.sum(((np.array(assey) - y_fit) / np.array(errors_y))**2)
dof = len(assey) - len(popt1)
rChiQ = ChiQ / dof
p_value = 1 - chi2.cdf(ChiQ, dof)

print(f"Chi quadro ridotto: {rChiQ:.2f}")
print(f"P-value: {p_value:.4f}")

Chi quadro ridotto: 1.91
P-value: 0.1054
